# The Purge

## Libraries and Settings

In [24]:
# Libraries
import re
import numpy as np
import pandas as pd
import json
import urllib
import urllib.parse
import requests
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Importing Data

In [25]:
# Read the data to a pandas data frame
df_complete = pd.read_csv('raw_data/coop_rot.csv', sep=',', encoding='utf-8')
df_weiss = pd.read_csv('raw_data/coop_weiss.csv', sep=',', encoding='utf-8')
df_complete = df_complete.append(df_weiss)
df_champagner = pd.read_csv('raw_data/coop_champagner.csv', sep=',', encoding='utf-8')
df_complete = df_complete.append(df_champagner)
df_dessert = pd.read_csv('raw_data/coop_dessert.csv', sep=',', encoding='utf-8')
df_complete = df_complete.append(df_dessert)
df_rose = pd.read_csv('raw_data/coop_rose.csv', sep=',', encoding='utf-8')
df_complete = df_complete.append(df_rose)
df_schaumweine = pd.read_csv('raw_data/coop_schaumweine.csv', sep=',', encoding='utf-8')
df_complete = df_complete.append(df_schaumweine)
df_complete

,web-scraper-order,typ,href,wein_name_raw,jahrgang_raw,genussreife_raw,land_raw,region_raw,produzent_raw,rebsorte_raw,alkohol_raw,inhalt_raw,preis_chf_raw
0,1671138041-4172,rot,https://www.coop.ch/de/weine/wein-sortiment/ro...,Apothic Inferno Red Blend Wine with a Whiskey ...,2018.0,2022-2023,USA,NaN,E. & J. Gallo Winery,NaN,16.00%,75cl,19.95
1,1671138044-4173,rot,https://www.coop.ch/de/weine/wein-sortiment/ro...,Apothic Dark California Red Blend,2016.0,2022-2024,USA,NaN,NaN,diverse Sorten,14.00%,75cl,12.95
2,1671138046-4174,rot,https://www.coop.ch/de/weine/wein-sortiment/ro...,Antonini Monte Chiara Montepulciano d'Abruzzo DOC,2020.0,2023-2024,Italien,Abruzzen,Montepulciano,NaN,12.50%,75cl,3.75
3,1671138049-4175,rot,https://www.coop.ch/de/weine/wein-sortiment/ro...,Angelin Langhe Nebbiolo DOC,2021.0,2022-2026,Italien,Piemont,Angelo Negro,Nebbiolo,14.00%,75cl,17.5
4,1671138052-4176,rot,https://www.coop.ch/de/weine/wein-sortiment/ro...,Amarone della Valpolicella DOCG Vigne Alte Zeni,2019.0,2024-2028,Italien,Venetien,Cantina Zeni,Molinara,15.50%,75cl,32.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,1671145310-6985,schaumwein,https://www.coop.ch/de/weine/wein-sortiment/sc...,Raphael Dal Bo Prosecco Superiore DOCG Millesi...,2021.0,2022-2025,Italien,Venetien,Raphael dal Bo,Glera,11.00%,75cl,11.95
169,1671145313-6987,schaumwein,https://www.coop.ch/de/weine/wein-sortiment/sc...,Prosecco Superiore DOCG Brut 1924 Carpène Malv...,NaN,2022-2023,Italien,Venetien,NaN,"diverse Sorten, \n Glera",11.00%,75cl,17.5
170,1671145316-6989,schaumwein,https://www.coop.ch/de/weine/wein-sortiment/sc...,Prosecco Rosé 0.15 Millesimato Brut,2021.0,2022-2024,Italien,Venetien,De Stefani,Pinot Noir,11.50%,75cl,16.5
171,1671145318-6991,schaumwein,https://www.coop.ch/de/weine/wein-sortiment/sc...,Prosecco Frizzante DOC Canti 75CL,NaN,NaN,NaN,NaN,NaN,NaN,0.00%,75cl,5.95


### Getting Data Types

In [26]:
df_complete.dtypes

web-scraper-order     object
typ                   object
href                  object
wein_name_raw         object
jahrgang_raw         float64
genussreife_raw       object
land_raw              object
region_raw            object
produzent_raw         object
rebsorte_raw          object
alkohol_raw           object
inhalt_raw            object
preis_chf_raw         object
dtype: object

# Extract and save relevant information from raw data using regular expressions (regex)

### Extract alcohol value

In [27]:
# Extract values from 'alkohol' strings
alkohol = []
for i in df_complete['alkohol_raw']:
    d1 = re.findall('(.*)\%', str(i))
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    alkohol.append(d2)

# Save as new variable in the pandas data frame
df_complete['alkohol'] = pd.Series(alkohol, dtype="float64")
    
# Print first 5 values
print(df_complete['alkohol_raw'].head(5), '\n')
print(df_complete['alkohol'].head(5))

0    16.00%
1    14.00%
2    12.50%
3    14.00%
4    15.50%
Name: alkohol_raw, dtype: object 

0    16.0
1    14.0
2    12.5
3    14.0
4    15.5
Name: alkohol, dtype: float64


### Get Content of bottles

In [28]:
# Extract values from 'inhalt_raw' strings
inhalt_cl = []
for i in df_complete['inhalt_raw']:
    chunks = re.split(' ', str(i))
    length = len(chunks)
    d1 = re.findall('(.*)cl', chunks[length-1])
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    inhalt_cl.append(d2)

# Save as new variable in the pandas data frame
df_complete['inhalt_cl'] = pd.Series(inhalt_cl, dtype="float64")
    
# Print first 5 values
print(df_complete['inhalt_raw'].head(5), '\n')
print(df_complete['inhalt_cl'].head(5))

0    75cl
1    75cl
2    75cl
3    75cl
4    75cl
Name: inhalt_raw, dtype: object 

0    75.0
1    75.0
2    75.0
3    75.0
4    75.0
Name: inhalt_cl, dtype: float64


### Get Number of bottles in a pack

In [29]:
# Extract amount of containers in a set from inhalt_raw
anzahl_fl = []
for i in df_complete['inhalt_raw']:
    d1 = re.findall('(.*)x', str(i))
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    if d2 == None:
        d2 = 1
    anzahl_fl.append(d2)

# Save as new variable in the pandas data frame
df_complete['anzahl_fl'] = pd.Series(anzahl_fl, dtype="Int64")
    
# Print first 5 values
print(df_complete['inhalt_raw'].head(5), '\n')
print(df_complete['anzahl_fl'].head(5))

0    75cl
1    75cl
2    75cl
3    75cl
4    75cl
Name: inhalt_raw, dtype: object 

0    1
1    1
2    1
3    1
4    1
Name: anzahl_fl, dtype: Int64


### Cleaning up the year of production

In [30]:
#Cleaning up and converting 'jahrgang_raw' to Int
jahrgang = []
for i in df_complete['jahrgang_raw']:
    if pd.notna(i):
        jahrgang.append(int(i))
    else:
        jahrgang.append(i)
df_complete['jahrgang'] = pd.Series(jahrgang, dtype="Int32")
print(df_complete['jahrgang'].head(5))

0    2018
1    2016
2    2020
3    2021
4    2019
Name: jahrgang, dtype: Int32


### Getting start and end date for primetime

In [31]:
# Extract values from 'genussreife_raw' strings
prime_start = []
for i in df_complete['genussreife_raw']:
    d1 = re.findall('(.*)-', str(i))
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    prime_start.append(d2)

# Save as new variable in the pandas data frame
df_complete['prime_start'] = pd.Series(prime_start, dtype="Int32")
    
# Print first 5 values
print(df_complete['genussreife_raw'].head(5), '\n')
print(df_complete['prime_start'].head(5))

0    2022-2023
1    2022-2024
2    2023-2024
3    2022-2026
4    2024-2028
Name: genussreife_raw, dtype: object 

0    2022
1    2022
2    2023
3    2022
4    2024
Name: prime_start, dtype: Int32


In [32]:
# Extract values from 'genussreife_raw' strings for prime_end
prime_end = []
for i in df_complete['genussreife_raw']:
    chunks = re.split('-', str(i))
    length = len(chunks)
    d1 = re.findall('\d\d\d\d', chunks[length-1])
    try:
        d2 = d1[0].strip().replace(',', '.')
    except:
        d2 = None
    prime_end.append(d2)

# Save as new variable in the pandas data frame
df_complete['prime_end'] = pd.Series(prime_end, dtype="Int64")
    
# Print first 5 values
print(df_complete['genussreife_raw'].head(5), '\n')
print(df_complete['prime_end'].head(5))

0    2022-2023
1    2022-2024
2    2023-2024
3    2022-2026
4    2024-2028
Name: genussreife_raw, dtype: object 

0    2023
1    2024
2    2024
3    2026
4    2028
Name: prime_end, dtype: Int64


### Cleaning up preis_chf_raw

In [33]:
# Extract values from 'genussreife_raw' strings for prime_end
price_per_cl = []
for i in df_complete['preis_chf_raw']:
    try:
        i = i.strip().replace('\'', '')
    except:
        i = None
    price_per_cl.append(i)

# Save as new variable in the pandas data frame
df_complete['preis_chf'] = pd.Series(price_per_cl, dtype="float64")
    
# Print first 5 values
print(df_complete['preis_chf'].head(5))

0    19.95
1    12.95
2     3.75
3    17.50
4    32.95
Name: preis_chf, dtype: float64


## Imputation of missing values in the columns prime_start and prime_end using mean()

### Updating missing values for prime start and prime end

In [34]:
for i in df_complete['typ']:
        difference = df_complete['prime_start'] - df_complete['jahrgang']
        mean_difference = int(difference.mean(skipna=True))
        
        # Fill in the null values in the 'prime_start' column with the mean difference
        if not df_complete['jahrgang'].empty:
            df_complete['prime_start'] = df_complete['prime_start'].fillna(df_complete['jahrgang'] + mean_difference)

        difference_prime = df_complete['prime_end'] - df_complete['prime_start']
        mean_difference_prime = int(difference.mean(skipna=True))
        if not df_complete['prime_start'].empty:
            df_complete['prime_end'] = df_complete['prime_end'].fillna(df_complete['prime_start'] + mean_difference)

df_complete = df_complete.dropna(subset=['jahrgang', 'prime_start', 'prime_end', 'land_raw'])

## Creating new variables 

### Creating new variable 'preis_cl'

In [35]:
df_complete['preis_cl'] = round(df_complete['preis_chf'] / (df_complete['anzahl_fl'] * df_complete['inhalt_cl']),2)

## Cleaning up the dataframe 

### New df for the cleaned data

In [36]:
# Create new df with the cleaned up data
df_cleaned = df_complete[['typ', 'wein_name_raw', 'land_raw', 'region_raw', 'produzent_raw', 'rebsorte_raw', 'alkohol', 'inhalt_cl', 'anzahl_fl', 'jahrgang', 'prime_start', 'prime_end', 'preis_chf', 'preis_cl']]

# Rename some of the values as to pertain the fact that it is now no longer raw data
df_cleaned.rename(columns = {'wein_name_raw':'wein_name', 'land_raw':'land', 'region_raw':'region', 'produzent_raw':'produzent', 'rebsorte_raw':'rebsorte'}, inplace = True)
df_cleaned

,typ,wein_name,land,region,produzent,rebsorte,alkohol,inhalt_cl,anzahl_fl,jahrgang,prime_start,prime_end,preis_chf,preis_cl
0,rot,Apothic Inferno Red Blend Wine with a Whiskey ...,USA,NaN,E. & J. Gallo Winery,NaN,16.0,75.0,1,2018,2022,2023,19.95,0.27
1,rot,Apothic Dark California Red Blend,USA,NaN,NaN,diverse Sorten,14.0,75.0,1,2016,2022,2024,12.95,0.17
2,rot,Antonini Monte Chiara Montepulciano d'Abruzzo DOC,Italien,Abruzzen,Montepulciano,NaN,12.5,75.0,1,2020,2023,2024,3.75,0.05
3,rot,Angelin Langhe Nebbiolo DOC,Italien,Piemont,Angelo Negro,Nebbiolo,14.0,75.0,1,2021,2022,2026,17.50,0.23
4,rot,Amarone della Valpolicella DOCG Vigne Alte Zeni,Italien,Venetien,Cantina Zeni,Molinara,15.5,75.0,1,2019,2024,2028,32.95,0.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,schaumwein,Raventós i Blanc Textures de Pedra,Spanien,Katalonien,Raventós i Blanc,Xarel-lo,13.3,75.0,1,2021,2022,2024,17.95,0.24
168,schaumwein,Raphael Dal Bo Prosecco Superiore DOCG Millesi...,Italien,Venetien,Raphael dal Bo,Glera,13.3,75.0,1,2020,2021,2024,24.95,0.33
169,schaumwein,Prosecco Superiore DOCG Brut 1924 Carpène Malv...,Italien,Venetien,NaN,"diverse Sorten, \n Glera",14.0,75.0,1,2020,2022,2025,18.50,0.25
170,schaumwein,Prosecco Rosé 0.15 Millesimato Brut,Italien,Venetien,De Stefani,Pinot Noir,15.0,75.0,1,2018,2022,2028,34.50,0.46


### Deleting duplicates

In [37]:
# Count duplicated values in the whole data set
print('Sum of missing values:', df_cleaned.duplicated().sum(), '\n')

# Identify duplicated values in 'rooms', 'area', 'price'
print('Duplicated values')
print(df_cleaned.loc[df_cleaned.duplicated(keep = 'last')])

# Drop the rows with duplicated values
df_cleaned = df_cleaned.drop_duplicates()

Sum of missing values: 1 

Duplicated values
    typ                                          wein_name        land  \
8  rose  J.P. Chenet Rosé Grenache-Cinsault Vin de Pays...  Frankreich   

                 region produzent rebsorte  alkohol  inhalt_cl  anzahl_fl  \
8  Languedoc-Roussillon  Grenache      NaN     15.0       75.0          1   

   jahrgang  prime_start  prime_end  preis_chf  preis_cl  
8      2019         2023       2026      24.95      0.33  


## Implementing latitude and longitude values using the openstreetmap Web API (Geocoding)

### Get list of all unique values in 'land'

In [38]:
df1 = df_cleaned[['land']]
df2 = df1.dropna()
df3 = df2.drop_duplicates()
df3 = df3.reset_index(drop=True)
df3.head()

,land
0,USA
1,Italien
2,Argentinien
3,Frankreich
4,Chile


### Get lat and lon values for each country via Web API

In [39]:
# create variables
geolocation = []
n = 1

# Iterate through each value and get lat and lon
for i in df3['land'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df3['land']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
            url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(i) +'?format=json'
            response = requests.get(url).json()
            geolocation.append(pd.Series(data={'lat': response[0]["lat"], 'lon': response[0]["lon"]}))

    except:
            geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
            
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df3['land'])))
df3['lat'] = df_loc['lat']
df3['lon'] = df_loc['lon']
df3.head(50)

,land,lat,lon
0,USA,39.7837304,-100.445882
1,Italien,42.6384261,12.674297
2,Argentinien,-34.9964963,-64.9672817
3,Frankreich,46.603354,1.8883335
4,Chile,-31.7613365,-71.3187697
5,Portugal,39.6621648,-8.1353519
6,Schweiz,46.7985624,8.2319736
7,Australien,-24.7761086,134.755
8,Spanien,39.3260685,-4.8379791
9,Deutschland,51.1638175,10.4478313


### Merge to existing df

In [40]:
# create new df and merge on lat and lon using the value land
merged_df = pd.merge(df_cleaned, df3, on='land')
for index, row in merged_df.iterrows():
    df_cleaned.loc[df_cleaned['land'] == row['land'], 'lat'] = row['lat']
    df_cleaned.loc[df_cleaned['land'] == row['land'], 'lon'] = row['lon']
df_cleaned.head()

,typ,wein_name,land,region,produzent,rebsorte,alkohol,inhalt_cl,anzahl_fl,jahrgang,prime_start,prime_end,preis_chf,preis_cl,lat,lon
0,rot,Apothic Inferno Red Blend Wine with a Whiskey ...,USA,NaN,E. & J. Gallo Winery,NaN,16.0,75.0,1,2018,2022,2023,19.95,0.27,39.7837304,-100.445882
1,rot,Apothic Dark California Red Blend,USA,NaN,NaN,diverse Sorten,14.0,75.0,1,2016,2022,2024,12.95,0.17,39.7837304,-100.445882
2,rot,Bike Scorpion mit Cabernet Sauvignon,USA,NaN,NaN,Cabernet Sauvignon,13.0,75.0,1,2019,2023,2027,42.95,0.57,39.7837304,-100.445882
3,rot,Cabernet Sauvignon Woodbridge Robert Mondavi,USA,NaN,Cabernet Sauvignon,NaN,13.5,75.0,1,2018,2022,2025,9.95,0.13,39.7837304,-100.445882
4,rot,Cabernet Sauvignon Valley Oaks California Fetzer,USA,NaN,Fetzer,Cabernet Sauvignon,13.5,75.0,1,2019,2023,2024,14.95,0.2,39.7837304,-100.445882


## Saving the data to csv

In [42]:
df_cleaned.to_csv('wine_data.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)